In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import sys
import scipy
import numpy
import json
import bokeh
import tensorflow
import keras
import sklearn
import matplotlib
import pandas as pd
import seaborn as sns

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [5]:
pip install findspark

In [6]:
import findspark
findspark.init()

In [7]:
import pyspark
from pyspark.sql import SparkSession
sp = pyspark.SparkContext()
spark = SparkSession(sp)
print(sp)

<SparkContext master=local[*] appName=pyspark-shell>


In [8]:
!pip install -U -q PyDrive # Pydrive helps in mouting dataset to collab

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials  #oauth2 support given
 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default() # prompt to sign in
drive = GoogleDrive(gauth)

In [9]:
file_list = drive.ListFile({'q': "'1fdHixmn1GUa_OXD3BQb5aJks8NQC2nHR' in parents and trashed=false"}).GetList() # folder_id posted here
for file1 in file_list:
  print('%s -> %s' % (file1['title'], file1['id'])) # List of files in the folder

dns.csv -> 1CAwm6CqvUZrVjouWpbdCu4uajFgk-Q-p
topd.csv -> 1Og7PpcAa1NfstsLdRL6efIYTDg8_lpvq
top.csv -> 1gAuYmaWRVcRfkkcBGFKtrK7iUzuIKW_y
dga.txt -> 1f1meAeoajwNXO-eUjnfk2FzsaERQIONI


In [10]:
f_id=''
drive_file = drive.CreateFile({'id': f_id})
drive_file = drive.CreateFile({'id':'1Og7PpcAa1NfstsLdRL6efIYTDg8_lpvq'}) #file_id to mount
drive_file.GetContentFile('topd.csv') #getfile

f_id=''
drive_file = drive.CreateFile({'id': f_id})
drive_file = drive.CreateFile({'id':'1CAwm6CqvUZrVjouWpbdCu4uajFgk-Q-p'}) #file_id to mount
drive_file.GetContentFile('dns.csv') #getfile

In [11]:
top_l=pd.read_csv("topd.csv",nrows=100000) #Read file
#print(top_l)
top_rdd = sp.parallelize(top_l['google.com'], 4)
#top_rdd.collect()

In [12]:
dga_p = pd.read_csv('dns.csv',nrows=100000)
print(dga_p)
dga_l = dga_p['dns1.viphone.cz.'].values.tolist()
dga_rdd = sp.parallelize(dga_p['dns1.viphone.cz.'], 4)
#dga_rdd.collect()

                               dns1.viphone.cz.
0             mtae448c7100586.mm.ks.ks.cox.net.
1                            ns1.mm.ks.cox.net.
2                                 24.248.134.39
3      236.Red-81-37-25.dynamicIP.rima-tde.net.
4                      rsdmno1-14.rima-tde.net.
...                                         ...
99994                              192.26.92.30
99995                              192.31.80.30
99996                              192.12.94.30
99997                              192.35.51.30
99998                              192.42.93.30

[99999 rows x 1 columns]


In [13]:
def filt(word):
    return ('[a-zA-Z0-9]*.' +word)

In [14]:
domain_filt_words = top_rdd.map(filt)
print(domain_filt_words.collect())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
pairkeynames = (domain_filt_words
                  .map(lambda na: (na, 1))
                  .reduceByKey(lambda o,p: o+p)
                  .collect())
print (pairkeynames)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
d_w = domain_filt_words.map(lambda na: (na, 1))

In [17]:
deptColumns = ["domain_name","count"]
df2 = d_w.toDF(deptColumns)
df2.printSchema()
df2.show(truncate=False)

p_df = df2.toPandas()
domain_df = p_df['domain_name']

domain_li = domain_df.values.tolist()
print(domain_li)

root
 |-- domain_name: string (nullable = true)
 |-- count: long (nullable = true)

+--------------------------+-----+
|domain_name               |count|
+--------------------------+-----+
|[a-zA-Z0-9]*.youtube.com  |1    |
|[a-zA-Z0-9]*.tmall.com    |1    |
|[a-zA-Z0-9]*.baidu.com    |1    |
|[a-zA-Z0-9]*.qq.com       |1    |
|[a-zA-Z0-9]*.sohu.com     |1    |
|[a-zA-Z0-9]*.facebook.com |1    |
|[a-zA-Z0-9]*.taobao.com   |1    |
|[a-zA-Z0-9]*.360.cn       |1    |
|[a-zA-Z0-9]*.jd.com       |1    |
|[a-zA-Z0-9]*.yahoo.com    |1    |
|[a-zA-Z0-9]*.amazon.com   |1    |
|[a-zA-Z0-9]*.sina.com.cn  |1    |
|[a-zA-Z0-9]*.wikipedia.org|1    |
|[a-zA-Z0-9]*.weibo.com    |1    |
|[a-zA-Z0-9]*.xinhuanet.com|1    |
|[a-zA-Z0-9]*.netflix.com  |1    |
|[a-zA-Z0-9]*.reddit.com   |1    |
|[a-zA-Z0-9]*.live.com     |1    |
|[a-zA-Z0-9]*.alipay.com   |1    |
|[a-zA-Z0-9]*.zhanqi.tv    |1    |
+--------------------------+-----+
only showing top 20 rows



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [25]:
import re
def reducer():
  to_rem = []
  for i in dga_l:
    for j in domain_li:
      emails = re.findall(j, i)
      for email in emails:
        to_rem.append(str(email))
  print(to_rem)
  return to_rem

In [26]:
to_rem = (non_rep_doms.map(lambda w: w).reduceBy(reducer()))

['ks.cox.net', 'ks.cox.net', 'dreamhost.co', 'dreamhost.com', 'ns3.dreamhost.com', 'dreamhost.co', 'dreamhost.com', 'ns1.dreamhost.com', 'dreamhost.co', 'dreamhost.com', 'ns2.dreamhost.com', 'mantraonline.com', 'mantraonline.com', 'hsd1.mi.com', 'mi.comcast.net', 'dns101.comcast.net', 'dns102.comcast.net', 'dns103.comcast.net', 'dns104.comcast.net', 'dns105.comcast.net', 'ns1.apnic.net', 'chinamobile.com', 'zj.chinamobile.com', 'chinamobile.com', 'zj.chinamobile.com', 'ns1.apnic.net', 'ns1.apnic.net', 'ctnt.co', 'ctnt.com', 'ns1.apnic.net', 'ns1.apnic.net', 'ns.ripe.net', 'colocrossing.com', 'colocrossing.com', 'audaxis.com', 'audaxis.com', 'arnet.co', 'arnet.co', 'classsicworldnet.co', 'classsicworldnet.co', '236.netcologne.de', 'seciu.edu', 'seciu.edu', 'broadriver.com', 'broadriver.com', 'dns101.comcast.net', 'ns1.apnic.net', 'host.co', 'colocrossing.com', 'tiete2.vivo.com', 'comprido.vivo.com', 'guandu.vivo.com', 'pinheiros2.vivo.com', 'res.rr.com', 'texas.rr.com', 'texas.rr.com', 

In [20]:
print(len(to_rem))

424


In [21]:
non_rep_doms = dga_rdd.filter(lambda x: x not in to_rem)
print(non_rep_doms.collect())

['mtae448c7100586.mm.ks.ks.cox.net.', 'ns1.mm.ks.cox.net.', '24.248.134.39', '236.Red-81-37-25.dynamicIP.rima-tde.net.', 'rsdmno1-14.rima-tde.net.', 'rsdbgi1-11.rima-tde.net.', 'COMPRIDO.VIVO.COM.BR.', 'PINHEIROS2.VIVO.COM.BR.', 'GUANDU.VIVO.COM.BR.', 'TIETE2.VIVO.COM.BR.', 'ns1.cnc-gd.net.', 'ns3.dreamhost.com.', 'ns1.dreamhost.com.', 'ns2.dreamhost.com.', 'dnsdel.mantraonline.com.', 'aaadel.mantraonline.com.', '202.56.230.6', '2404:a800:0:b::9', '202.56.230.5', '2404:a800:0:b::7', 'ns1.odessa.comstar.net.ua.', 'ns2.odessa.comstar.net.ua.', 'apache2-ps42489.ps42489.dreamhostps.com.', 'c-69-246-18-237.hsd1.mi.comcast.net.', 'dns101.comcast.net.', 'dns102.comcast.net.', 'dns103.comcast.net.', 'dns104.comcast.net.', 'dns105.comcast.net.', 'ns.golden.net.', 'ns2.golden.net.', 'ns1.apnic.net.', '211.138.130.254', '85-23-50-13.bb.dnainternet.fi.', '211.140.14.36', 'ns1.zj.chinamobile.com.', 'ns1.zj.chinamobile.com.', 'ns3.epidc.co.kr.', 'ns2.epidc.co.kr.', 'ns1.apnic.net.', 'ns1.apnic.net.'

In [22]:
uniq_doms = (non_rep_doms.map(lambda w: (w, 1)).reduceByKey(lambda x,y: x+y))
print(uniq_doms.collect())

[('mtae448c7100586.mm.ks.ks.cox.net.', 1), ('rsdbgi1-11.rima-tde.net.', 160), ('COMPRIDO.VIVO.COM.BR.', 62), ('ns1.dreamhost.com.', 4), ('dnsdel.mantraonline.com.', 37), ('2404:a800:0:b::9', 36), ('ns2.odessa.comstar.net.ua.', 2), ('dns101.comcast.net.', 368), ('dns103.comcast.net.', 240), ('dns104.comcast.net.', 240), ('211.138.130.254', 31), ('211.140.14.36', 31), ('199.166.6.5', 10), ('212.113.37.157', 1), ('ns2.ukrtelecom.ua.', 5), ('195.5.6.10', 3), ('193.189.231.2', 30), ('193.189.231.194', 30), ('ns1.tpgi.com.au.', 5), ('gulp.arnet.com.ar.', 15), ('g.gtld-servers.net.', 472), ('d.gtld-servers.net.', 472), ('b.gtld-servers.net.', 472), ('m.gtld-servers.net.', 472), ('a.gtld-servers.net.', 520), ('h.gtld-servers.net.', 472), ('192.12.94.30', 506), ('192.54.112.30', 505), ('192.48.79.30', 505), ('192.41.162.30', 528), ('j.root-servers.net.', 349), ('g.root-servers.net.', 354), ('l.root-servers.net.', 354), ('f.root-servers.net.', 354), ('192.228.79.201', 338), ('2001:500:2d::d', 31

In [27]:
print(uniq_doms)

PythonRDD[22] at collect at <ipython-input-22-9c6f1d2e4c00>:2


In [28]:
deptColumns = ["domain_name","count"]
fin = uniq_doms.toDF(deptColumns)
fin.printSchema()
fin.show(truncate=False)

fin_df = fin.toPandas()
domainf_df = fin_df['domain_name']

final_domain_li = domain_df.values.tolist()
print(final_domain_li)

root
 |-- domain_name: string (nullable = true)
 |-- count: long (nullable = true)

+---------------------------------+-----+
|domain_name                      |count|
+---------------------------------+-----+
|mtae448c7100586.mm.ks.ks.cox.net.|1    |
|rsdbgi1-11.rima-tde.net.         |160  |
|COMPRIDO.VIVO.COM.BR.            |62   |
|ns1.dreamhost.com.               |4    |
|dnsdel.mantraonline.com.         |37   |
|2404:a800:0:b::9                 |36   |
|ns2.odessa.comstar.net.ua.       |2    |
|dns101.comcast.net.              |368  |
|dns103.comcast.net.              |240  |
|dns104.comcast.net.              |240  |
|211.138.130.254                  |31   |
|211.140.14.36                    |31   |
|199.166.6.5                      |10   |
|212.113.37.157                   |1    |
|ns2.ukrtelecom.ua.               |5    |
|195.5.6.10                       |3    |
|193.189.231.2                    |30   |
|193.189.231.194                  |30   |
|ns1.tpgi.com.au.                 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

